In [1]:
# 事前にHugging FaceでアカウントとTokenを取得しておく
# https://huggingface.co/settings/tokens
from huggingface_hub import notebook_login


notebook_login()


In [13]:
# google/gemma-2-2b-jpn-it モデルを使って日本語の文章生成を行う場合
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-jpn-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="mps",
)
prompt = "LLMについて教えて下さい。"
outputs = pipe(prompt, return_full_text=False, max_new_tokens=512)
response = outputs[0]["generated_text"].strip()
print(response)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

**LLM** は、**Large Language Model** の略称です。

**LLMの特徴**

* **大量のテキストデータで学習:**  膨大な量のテキストデータから学習することで、人間のような自然な言語を理解し、生成することができます。
* **多様なタスクに対応:**  文章の要約、翻訳、質問応答、文章生成など、様々なタスクに対応できます。
* **人間と自然な対話:**  人間と自然な会話ができるようになり、より人間らしいコミュニケーションが可能になりました。

**LLMの応用**

* **Chatbot:**  顧客サービスやサポートに活用できます。
* **翻訳:**  多言語翻訳が可能になり、国際的なコミュニケーションが容易になりました。
* **文章生成:**  ブログ記事、小説、詩など、様々な文章を作成できます。
* **情報検索:**  より自然な検索方法を提供できます。
* **教育:**  学習教材や質問応答システムとして活用できます。


**LLMの課題**

* **偏見:**  学習データに偏りが存在する場合、偏った結果が出ることがあります。
* **倫理的な問題:**  生成された文章が事実と異なる場合や、差別的な内容が含まれる可能性があります。
* **セキュリティ:**  LLMのモデルを悪用して、虚偽の情報や攻撃的なコンテンツを生成する可能性があります。



**まとめ**

LLMは、人間のような自然な言語を理解し、生成できる強力なAI技術です。様々な分野で応用が期待されていますが、同時に課題も存在します。


In [1]:
# 最初に実行すると~/.cache/huggingface/hubにキャッシュが保存されるので遅いが
# 次回からはキャッシュを使って高速に実行できるので速い
from mlx_lm import generate, load

model, tokenizer = load("mlx-community/gemma-2-2b-jpn-it")
prompt = "LLMについて教えてください。"
response = generate(model, tokenizer, prompt=prompt, verbose=True, max_tokens=512)


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Prompt: LLMについて教えてください。


**LLM** は、**Large Language Model** の略称です。

**LLMの特徴**

* **大量のテキストデータで学習:**  膨大な量のテキストデータから学習することで、人間のような自然な言語を理解し、生成することができます。
* **多様なタスクに対応:**  文章の要約、翻訳、質問応答、文章生成など、様々なタスクに対応できます。
* **人間と自然な対話:**  人間と自然な会話ができるようになり、より人間らしいコミュニケーションが可能になりました。

**LLMの応用**

* **Chatbot:**  顧客サービスやサポートに活用できます。
* **翻訳:**  多言語翻訳が可能になり、国際的なコミュニケーションが容易になりました。
* **文章生成:**  ブログ記事、小説、詩など、様々な文章を作成できます。
* **情報検索:**  より自然な検索方法を提供できます。
* **教育:**  学習教材や質問応答システムとして活用できます。


**LLMの課題**

* **偏見:**  学習データに偏りが存在する場合、偏った結果が出ることがあります。
* **倫理的な問題:**  生成された文章が事実と異なる場合や、差別的な内容が含まれる可能性があります。
* **セキュリティ:**  LLMのモデルを悪用して、虚偽の情報や悪意のあるコンテンツを生成する可能性があります。



**まとめ**

LLMは、人間のような自然な言語を理解し、生成できる強力な技術です。様々な分野で応用が期待されていますが、同時に課題も存在します。


<end_of_turn>
Prompt: 7 tokens, 8.066 tokens-per-sec
Generation: 336 tokens, 6.116 tokens-per-sec
Peak memory: 7.303 GB


In [63]:
# https://qiita.com/jw-automation/items/045917be7b558509fdf2から引用
document = """
ここまでRAGの拡張手法についてまとめてきましたが、実際にRAGを進める際は、とにかく実装を頑張れば良いという訳ではなく、以下の3つを分けて考える必要があります。
①ユーザークエリに問題がないか
②参照ドキュメントに問題がないか
③RAGの実装に問題がないか
そもそも論として、AIがやっているのはインプットを元にしたアウトプットへの変換処理なので、インプットが悪ければ良いアウトプットは出せません。つまり、ユーザーから渡されたクエリと参照ドキュメント自体の質が悪ければ、どれだけAIが賢くても良いアウトプットは出せないという事です。あなたがどれだけ賢かったとしても、よくわからない指示と、誰もメンテしていないマニュアルを渡された場合に上手く仕事を進められないのと同じ話です。つまり、プロトタイプで上手く行かなかった場合は、①クエリの問題なのか、②ドキュメントの問題なのか、③RAGの実装の問題なのかを見極めながら対処していく必要があります。簡単なチェックポイントとしては、「自分がそのクエリとドキュメントを渡されて回答できるか？」という事を考えてみるのが良いのではないかと思います。自分でも回答できないのであればそもそものインプットが悪いという話になりますし、自分なら回答できるという事であれば、検索と応答生成の問題のため、RAGの実装を見直した方が良いでしょう。インプット自体が悪い状態でRAGの実装をひたすら頑張るのは、大量のノイズや欠損データを含むビッグデータを渡されたエンジニアが、ディープラーニングのチューニングだけをひたすら頑張るのと同じ話で(結局精度が出ないので)、まず見直すべきは上流のインプットになります。ユーザークエリの改善については、自由入力をやめてみるというのも1つポイントになるかと思います。というのも、自由入力は必ずしもユーザビリティが高くないからです。例えば、AIアシスタントに「今日は何が食べたいですか？何でも言って下さい」と言われるより、「今日は何が食べたいですか？イタリアンであれば、近くに有名な〇〇のお店がありすぐに予約が取れます。中華料理であれば～～、和食であれば～～」というように、何でも聞いてという形より、提案型で、ある種考える範囲を絞ってもらうほうが楽という場合もあると思います。つまり、自由入力で全てをユーザー側に考えさせるのではなく、適切に誘導させながら対象範囲を絞っていくという形にする事で、ユーザーも、そのインプットを受け取るAIも楽になる可能性があります。自由入力のインプット枠だけ用意して、「意図がよくわかるように詳細に入力して下さい」とユーザーにお願いするのは無理があるので、プルダウンや聞き返す等のアクションも利用しながら、AIに入る前の入力の質を上げる設計も重要な考慮ポイントになります。ドキュメントの改善については、クレンジングの観点で、AIに校正させるという事も一つの有効な手になります。誤字脱字や重複箇所、記載内容が不明瞭な箇所や、複数のドキュメントで記載内容が矛盾している箇所などのチェックを全て人手でやるのは難しいので、校正の補助プロセスでAIを使い、クレンジングしてからインデックス化するというのが、まずはそれほど手間をかけずに実施できる施策かと思います。プログラミングにおけるコーディングチェックツールのような位置付けで、ドキュメントの自動チェックをパイプライン化してしまうと尚良いかもしれません。なかなか一筋縄にはいかないですが、端的に言ってしまうと、ドキュメント全体がMECEになっていれば、ベクトル検索で問題なく取得できるはずで、優秀な人が作る、誰にとっても読みやすいドキュメント(構成の綺麗なドキュメント)は、AIにとっても読みやすいと言えます。いかにAIにコンテキスト情報を渡すかという事が論点になっていく世界感においては、そのベースとなるナレッジマネジメントがある種最も重要になるかもしれません。いずれにしても、エンジニアに実装だけ丸投げしても上手くいかないので、「どこがボトルネックなのか？」という事を見極めて、プロジェクトとして対処していく必要があります。
"""


prompt = "以下の文章について箇条書きで要約してください。\n" + document

response = generate(model, tokenizer, prompt=prompt, verbose=True, max_tokens=1000)


Prompt: 以下の文章について箇条書きで要約してください。

ここまでRAGの拡張手法についてまとめてきましたが、実際にRAGを進める際は、とにかく実装を頑張れば良いという訳ではなく、以下の3つを分けて考える必要があります。
①ユーザークエリに問題がないか
②参照ドキュメントに問題がないか
③RAGの実装に問題がないか
そもそも論として、AIがやっているのはインプットを元にしたアウトプットへの変換処理なので、インプットが悪ければ良いアウトプットは出せません。つまり、ユーザーから渡されたクエリと参照ドキュメント自体の質が悪ければ、どれだけAIが賢くても良いアウトプットは出せないという事です。あなたがどれだけ賢かったとしても、よくわからない指示と、誰もメンテしていないマニュアルを渡された場合に上手く仕事を進められないのと同じ話です。つまり、プロトタイプで上手く行かなかった場合は、①クエリの問題なのか、②ドキュメントの問題なのか、③RAGの実装の問題なのかを見極めながら対処していく必要があります。簡単なチェックポイントとしては、「自分がそのクエリとドキュメントを渡されて回答できるか？」という事を考えてみるのが良いのではないかと思います。自分でも回答できないのであればそもそものインプットが悪いという話になりますし、自分なら回答できるという事であれば、検索と応答生成の問題のため、RAGの実装を見直した方が良いでしょう。インプット自体が悪い状態でRAGの実装をひたすら頑張るのは、大量のノイズや欠損データを含むビッグデータを渡されたエンジニアが、ディープラーニングのチューニングだけをひたすら頑張るのと同じ話で(結局精度が出ないので)、まず見直すべきは上流のインプットになります。ユーザークエリの改善については、自由入力をやめてみるというのも1つポイントになるかと思います。というのも、自由入力は必ずしもユーザビリティが高くないからです。例えば、AIアシスタントに「今日は何が食べたいですか？何でも言って下さい」と言われるより、「今日は何が食べたいですか？イタリアンであれば、近くに有名な〇〇のお店がありすぐに予約が取れます。中華料理であれば～～、和食であれば～～」というように、何でも聞いてという形より、提案型で、ある種考える範囲を絞ってもらうほうが楽という場合もあると思います。つまり